# Gradient descent


Suposse that we have $N$ observation of a vector $\mathbf{x}_i\in\mathbb{R}^D$ and a output varibale $t_i\in\mathbb{R}$. We want a model of the form $t_i=w_0+\sum_{j=1}^D w_j x_{ij}$. It is possible to construct a  cost function $E(\mathbf{w})$ define as:

$$
\begin{equation}
E(\mathbf{w})=\frac{1}{2N}\sum_{i=1}^N\left(t_n - (w_0 + \sum_{j=1}^D w_j (x_{i})_j \right)^2
\end{equation}
$$
That we could write as:
$$
\begin{equation}
E(\mathbf{w})=\frac{1}{2N}\sum_{i=1}^N\left(t_n - (w_0 + \mathbf{w}^T\mathbf{x}_n) \right)^2
\end{equation}
$$

Or in matrix form:
$$
\begin{equation}
E(\mathbf{w})=\frac{1}{2N}\left(\mathbf{t} - \mathbb{X}\tilde{\mathbf{w}} \right)^T\left(\mathbf{t} - \mathbb{X}\tilde{\mathbf{w}} \right),
\end{equation}  
$$
where
$$
\tilde{\mathbf{w}}=\begin{bmatrix}w_0 \\ \mathbf{w} \end{bmatrix},\;\;\;\;\mathbb{X}=\begin{bmatrix}
1&&\mathbf{x_1} \\
1&&\mathbf{x_2} \\
\vdots && \vdots \\
1&&\mathbf{x_N}
\end{bmatrix},\;\;\;\mathbf{t}=\begin{bmatrix}t_1\\t_2\\ \vdots \\ t_N\end{bmatrix}.
$$

The optimal solution in Less squares form is given by:
$$
\begin{equation}
\tilde{\mathbf{w}}^* = \left( \mathbb{X}^T\mathbb{X} \right)^{-1}\mathbb{X}^T\mathbf{y}
\end{equation}
$$

In [ ]:
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("LaTeXStrings")
import Pkg; Pkg.add("ColorSchemes")
using LinearAlgebra
using Plots
using LaTeXStrings
using ColorSchemes
pyplot()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed PyPlot ───── v2.8.2
 Installed PyCall ───── v1.91.2
 Installed MacroTools ─ v0.5.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [d330b81b] + PyPlot v2.8.2
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [1914dd2f] + MacroTools v0.5.2
  [438e738f] + PyCall v1.91.2
  [d330b81b] + PyPlot v2.8.2
  Building PyCall → `~/.julia/packages/PyCall/ttONZ/deps/build.log`


In [ ]:
# The data to fit
N = 30
𝚠 = [0.5, 1.0] # True weights
x = collect(range(-1.0 ,stop = 1.0, length=N))
t = [ones(N,1) x] * 𝚠

𝚇 = [ones(N,1) x]

# Less squares solution
wo = inv(𝚇'*𝚇)*(𝚇'*t)

In [2]:
scatter(x, t, label="Data")
xaxis!(L"x")
yaxis!(L"y")

# Plot less squares solution
tls = [ones(N,1) x] * wo
plot!(x, tls, label =L"$y=w_0+\mathbf{w}^T\mathbf{x}$" )

UndefVarError: UndefVarError: scatter not defined

Insted of use the Less Squares solution, we are going to use a iterative method, the Gradient Step Descent (SG), the update rule is:

$$
\begin{equation}
\mathbf{w}^{i+1}=\mathbf{w}^{i}-\alpha\nabla E(\mathbf{w}^{i})    
\end{equation}    
$$, where
$i$ indicates the iteration and $\alpha$ is calling the **learning rate**.

In our case $\nabla E(\mathbf{w})$ is given by:

$$
\begin{eqnarray}
\nabla E(\mathbf{w})&=&\frac{1}{N}\left[-\mathbb{X}^T\mathbf{t} + (\mathbb{X}^T \mathbb{X}\mathbf{w}) \right] \\
&=&\frac{1}{N}\mathbb{X}^T\left[ \mathbb{X}\mathbf{w} - \mathbf{t}\right]
\end{eqnarray}.
$$

Due is a iterative method, we need a stop criteria; the most common criteria ins this algorithms are a maximum number of iterations, or $||\nabla E(\mathbf{w} ||\leq \epsilon$.

In [3]:

struct Parameters
    # Parameters
    maxIters::Int64
    α::Float64
    ϵ::Float64
end


# y(xᵢ) = w₀ + w₁x
function cost(x::Vector{Float64}, w::Vector{Float64}, y::Vector{Float64})

    diff   = 𝚇*w - t # t - 𝚇*w

    val     = (1.0/(2.0*N)) * (diff' * diff)
    grad    = (1.0/N)*(𝚇'*(diff)) #(-𝚇'*t + (𝚇' * 𝚇 * w) )

    return val, grad
end


function stepGradient!(w₀::Vector{Float64}, params::Parameters,
    gradNorm::Vector{Float64},
    f_cost::Vector{Float64},
    m_∇f::Matrix{Float64},
    ws::Matrix{Float64})

        local w = w₀
        α       = params.α
        it      = 0
        for i = 1:params.maxIters

            ws[:,i] = w # Just for plots

            c, ∇f = cost(x, w, y)
            wᵢ = w - α * ∇f
            w  = wᵢ

            # Copy data for plots
            gradNorm[i] = norm(∇f)
            fcost[i]    = c
            m_∇f[:,i]   = -α*∇f

            it = i
            if norm(∇f) <= params.ϵ
                it = i
                break
            end
        end
        return w, it
 end


p       = Parameters(100, 0.6, 1.0e-6)
iters   = 0
normG   = zeros(Float64, p.maxIters)
fcost   = zeros(Float64, p.maxIters)
m∇f     = zeros(Float64, 2, p.maxIters)
ws      = zeros(Float64, 2, p.maxIters)

wSG, iters  = stepGradient!([3.0, 3.0], p, normG, fcost, m∇f, ws)


UndefVarError: UndefVarError: x not defined

In [4]:
yls = [ones(N,1) x] * wSG

p0 = scatter(x, y, label="Data")
plot!(x, yls, label=L"$y=w_0+\mathbf{w}^T\mathbf{x}$-SG")
xaxis!(L"x")
yaxis!(L"y")

p1 = plot(fcost[1:iters], color="red", label="cost")
xaxis!("iters")
yaxis!(L"E(w)")

p2 = plot(normG[1:iters], color="green", label="")
xaxis!("iters")
yaxis!(L"||\nabla E(w)||")

plot(p0, p1,p2)

UndefVarError: UndefVarError: N not defined

In [5]:
w0 = -3.2:0.5:4
w1 = -2:0.5:4

f(xx::Float64, yy::Float64) = begin
    diff   = t - 𝚇*[xx,yy]
    val     = (1.0/(2.0*N)) * (diff' * diff)
    end
p3  = contour(w0, w1, f, fill=true, seriescolor = cgrad(ColorSchemes.viridis.colors))
xaxis!(L"w_0")
yaxis!(L"w_1")
quiver!(ws[1,1:iters], ws[2,1:iters], quiver=(m∇f[1,1:iters],m∇f[2,1:iters]), color="red")
plot(p3)

UndefVarError: UndefVarError: ColorSchemes not defined